In [2]:
import numpy as np
import random

# pretty print for np.array 
# from https://stackoverflow.com/questions/53126305/pretty-printing-numpy-ndarrays-using-unicode-characters/53164538#53164538
def pretty_print(A):
    if A.ndim==1:
        print(A)
    else:
        w = max([len(str(s)) for s in A]) 
        print(u'\u250c' + u'\u2500' * w + u'\u2510') 
        for AA in A:
            print(' ', end='')
            print('[', end='')
            for i, AAA in enumerate(AA[:-1]):
                w1 = max([len(str(s)) for s in A[:, i]])
                print(str(AAA) + ' ' * (w1  - len(str(AAA)) + 1), end='')
            w1 = max([len(str(s)) for s in A[:, -1]])
            print(str(AA[-1]) + ' ' * (w1 - len(str(AA[-1]))), end='')
            print(']')
        print(u'\u2514'+u'\u2500' * w + u'\u2518')  

def compare(left: set, right: set):
    if len(left) == len(right):
        for i in range(len(left)):
            if left[i] != right[i]:
                return left[i] < right[i]
    return left < right


### Generate $\mathcal{A}_n^2$

> Let $|\cdot|$ denotes cardinality and $\Delta$ denote symmetric different

In [3]:
def Delta(left: set, right: set):
    return left.symmetric_difference(right)

def Cardi(x: set):
    return len(x)

def legal(alpha_i: set, alpha_i_1: set):
    return Cardi(alpha_i) <= Cardi(alpha_i_1) and len(Delta(alpha_i, alpha_i_1)) <= 2

> Let $n=2^m$, $\Omega$ be a set of $m$ element such that $|\alpha_i| \leq |\alpha_{i+1}|$ and $|\alpha_i \Delta \alpha_{i+1}| \leq 2$  
> Let $\alpha_0=\{\varnothing\}$, now genereate $\Omega$

In [11]:
from itertools import chain, combinations

# param
m = 4
# generate \Omega

def create_Omega(_m):
    s = list(range(1, _m + 1))
    container = chain.from_iterable(combinations(s, r) for r in range(len(s)+1))
    map = {i: [] for i in range(1, len(s) + 1)}
    for item in container:
        if len(item) != 0:
            map[len(item)].append(set(item))
    res = [set()]
    for key, value in map.items():
        if key % 2 == 0:
            value.reverse()
        res.extend(value)
    return res

def assert_omega(ome, _m):
    for i in range(_m - 1):
        assert(legal(ome[i], ome[i + 1]))
    assert(len(ome) == 2**_m)

omega = create_Omega(m)
assert_omega(omega, m)
omega

[set(),
 {1},
 {2},
 {3},
 {4},
 {3, 4},
 {2, 4},
 {2, 3},
 {1, 4},
 {1, 3},
 {1, 2},
 {1, 2, 3},
 {1, 2, 4},
 {1, 3, 4},
 {2, 3, 4},
 {1, 2, 3, 4}]

> $$a_{ij} = \begin{cases}
    -1,\;\alpha_j\bigcap(\alpha_{i-1}\bigcup\alpha_i)=\alpha_{i-1}\Delta\alpha_{i}\;and\;|\alpha_{i-1}\Delta\alpha_{i}|=2 \\
    (-1)^{|\alpha_{i-1}\bigcap\alpha_j| + 1},\;\alpha_{j}\bigcap(\alpha_{i-1}\bigcup\alpha_{i})\neq\varnothing\;but\;does\;not\;meet\;the\;condition\;above \\
    1,\;\alpha_j\bigcap(\alpha_{i-1}\bigcup\alpha_i)=\varnothing \\
\end{cases}
$$

In [7]:
def query_element(i: int, j: int, _omega: list) -> int:
    alpha_j = _omega[j]
    alpha_i_1 = _omega[i - 1]
    alpha_i = _omega[i]

    if alpha_j.intersection(alpha_i_1.union(alpha_i)) == Delta(alpha_i_1, alpha_i) \
        and Cardi(Delta(alpha_i_1, alpha_i)) == 2:
        return -1
    elif Cardi(alpha_j.intersection(alpha_i_1.union(alpha_i))) != 0:
        return (-1)**(Cardi(alpha_i_1.intersection(alpha_j)) + 1)
    elif Cardi(alpha_j.intersection(alpha_i_1.union(alpha_i))) == 0:
        return 1
    else:
        raise ValueError("Undefined behavior!")
    
def create_A(_omega, _m):
    A_mat = np.zeros((2**_m, 2**_m))
    for i in range(A_mat.shape[0]):
        for j in range(A_mat.shape[1]):
            A_mat[i, j] = query_element(i, j, _omega)
    return A_mat

A_mat = create_A(omega, m)
pretty_print(A_mat.astype(int))

┌─────────────────────────────────────────────────┐
 [1 1  1  1  1  -1 -1 -1 -1 -1 -1 1  1  1  1  -1]
 [1 -1 1  1  1  1  1  1  -1 -1 -1 -1 -1 -1 1  -1]
 [1 1  -1 1  1  1  -1 -1 1  1  -1 -1 -1 1  -1 -1]
 [1 1  1  -1 1  -1 1  -1 1  -1 1  -1 1  -1 -1 -1]
 [1 1  1  1  -1 -1 -1 1  -1 1  1  1  -1 -1 -1 -1]
 [1 1  1  -1 1  1  1  -1 1  -1 1  -1 1  1  1  1 ]
 [1 1  -1 1  1  -1 1  -1 1  1  -1 -1 1  -1 -1 -1]
 [1 1  1  -1 1  -1 -1 1  1  -1 1  1  -1 -1 -1 -1]
 [1 -1 1  1  -1 1  1  -1 -1 1  1  -1 1  1  -1 -1]
 [1 1  1  -1 1  -1 1  -1 -1 1  1  1  -1 -1 -1 -1]
 [1 1  -1 1  1  1  -1 -1 1  -1 1  -1 1  -1 -1 -1]
 [1 1  1  -1 1  -1 1  1  1  1  -1 -1 -1 1  1  -1]
 [1 1  1  1  -1 -1 1  -1 1  -1 -1 1  -1 -1 -1 1 ]
 [1 1  1  -1 1  1  -1 -1 -1 1  -1 -1 1  -1 -1 1 ]
 [1 1  -1 1  1  -1 1  1  -1 -1 -1 -1 -1 1  -1 1 ]
 [1 -1 1  1  1  -1 -1 -1 1  1  1  -1 -1 -1 1  1 ]
└─────────────────────────────────────────────────┘


### Generate $\mathcal{A}_{n - 1}^1$

> Let $B\in\mathcal{A}_{n - 1}^1$, $\Phi(B)\in\mathcal{A}_{n}^2\;$:
> $$ \Phi(B) = \left(\begin{array}{cc} 
1 & 1_{n-1}\\
-1^T_{n-1} & 2B-J_{n-1}
\end{array}\right)
> $$
> And: 
> $$ \Phi(B)^{-1} = \left(\begin{array}{cc} 
1-\frac{1}{2}1_{n-1}B^{-1}1^T_{n-1} & -\frac{1}{2}1_{n-1}B^{-1}\\
\frac{1}{2}B^{-1}1^T_{n-1} & \frac{1}{2}B^{-1}
\end{array}\right)
> $$

In [8]:
def create_A_1_mat(A_2_mat):
    A_1_mat = A_2_mat.copy()
    for i in range(1, A_1_mat.shape[0]):
        A_1_mat[i, :] = A_1_mat[i, :] * -1

    for j in range(1, A_1_mat.shape[1]):
        if A_1_mat[0, j] == -1:
            A_1_mat[:, j] = A_1_mat[:, j] * -1

    A_1_mat = (A_1_mat[1:, 1:] + np.ones(2**m - 1)) * 0.5
    return A_1_mat

pretty_print(create_A_1_mat(A_mat).astype(int))

┌───────────────────────────────┐
 [1 0 0 0 1 1 1 0 0 0 1 1 1 0 0]
 [0 1 0 0 1 0 0 1 1 0 1 1 0 1 0]
 [0 0 1 0 0 1 0 1 0 1 1 0 1 1 0]
 [0 0 0 1 0 0 1 0 1 1 0 1 1 1 0]
 [0 0 1 0 1 1 0 1 0 1 1 0 0 0 1]
 [0 1 0 0 0 1 0 1 1 0 1 0 1 1 0]
 [0 0 1 0 0 0 1 1 0 1 0 1 1 1 0]
 [1 0 0 1 1 1 0 0 1 1 1 0 0 1 0]
 [0 0 1 0 0 1 0 0 1 1 0 1 1 1 0]
 [0 1 0 0 1 0 0 1 0 1 1 0 1 1 0]
 [0 0 1 0 0 1 1 1 1 0 1 1 0 0 0]
 [0 0 0 1 0 1 0 1 0 0 0 1 1 1 1]
 [0 0 1 0 1 0 0 0 1 0 1 0 1 1 1]
 [0 1 0 0 0 1 1 0 0 0 1 1 0 1 1]
 [1 0 0 0 0 0 0 1 1 1 1 1 1 0 1]
└───────────────────────────────┘


### Verify $χ(A')=2^{\frac{1}{2}nlogn - n(1+o(1))}$

In [14]:
from math import log

def funcX(A):
    return np.matrix(A).I.max()

m = 6
omega = create_Omega(m)
assert_omega(omega, m)
A_mat = create_A(omega, m)
A_1_mat = create_A_1_mat(A_mat)
X_A = funcX(A_1_mat)
n = A_1_mat.shape[0]
o_1 = 0.5 * n * log(n) - n - log(X_A)
print(o_1)

66.73528539117935
